In [1]:
import pandas as pd
import pickle as p
import numpy as np
import operator

from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords

In [2]:
train_data = pd.read_csv('../dataset/train.csv').fillna('')
test_data = pd.read_csv('../dataset/test.csv').fillna('')

## Get actual lengths of toxic etc

In [3]:
toxic_comments = train_data.comment_text.iloc[train_data.toxic.nonzero()]
severe_toxic_comments = train_data.comment_text.loc[train_data.severe_toxic.nonzero()]
obscene_comments = train_data.comment_text.loc[train_data.obscene.nonzero()]
threat_comments = train_data.comment_text.loc[train_data.threat.nonzero()]
insult_comments = train_data.comment_text.loc[train_data.insult.nonzero()]
identity_hate_comments = train_data.comment_text.loc[train_data.identity_hate.nonzero()]

In [ ]:
train_data.comment_text.str.split().str.len().describe()

## Toxic

In [ ]:
toxic_comment_lengths = toxic_comments.str.split().str.len()

In [ ]:
toxic_comment_lengths.describe()

In [ ]:
%matplotlib inline
toxic_comment_lengths.plot(kind='hist')

## Severe

In [ ]:
severe_comment_lengths = severe_toxic_comments.str.split().str.len()

In [ ]:
severe_comment_lengths.describe()

## Obscene

In [ ]:
obscene_comment_lengths = obscene_comments.str.split().str.len()

In [ ]:
obscene_comment_lengths.describe()

## Threat

In [ ]:
threat_comment_lengths = threat_comments.str.split().str.len()

In [ ]:
threat_comment_lengths.describe()

## Insult

In [ ]:
insult_comment_lengths = insult_comments.str.split().str.len()

In [ ]:
insult_comment_lengths.describe()

## Identity 

In [ ]:
identity_comment_lengths = identity_hate_comments.str.split().str.len()

In [ ]:
insult_comment_lengths.describe()


The mean for all the targets are less than 100 so maybe turn down the sequence length...

In [4]:
import textacy
list(textacy.extract.ngrams(
...     train_data.comment_text[0], 2, filter_stops=True, filter_punct=True, filter_nums=False))

SyntaxError: invalid syntax (<ipython-input-4-8b6ce49a86f0>, line 3)

In [ ]:
from textacy.doc import Doc

comments = list(map(lambda doc: Doc(content=doc, lang='en'), train_data.comment_text))

### Remove non english chars and special chars

In [ ]:
stop_words = set(stopwords.words('english')).union(STOP_WORDS)
custom_stopwords = set(['wikipedia', 'article', 'page', 'talk', 'like', 'know', 'edit', 'use', 
                        'think', 'wiki', 'people', 'b', 'hi', 'hey', ])
stop_words = stop_words.union(custom_stopwords)

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(filters='')

In [ ]:
tokenizer.fit_on_texts(train_data.comment_text.astype(str))

In [ ]:
tokenizer.word_index['!!!!']

In [ ]:
import re

remove = re.compile('[^a-zA-Z ]')

def filter_non_english_chars(documents):
    return pd.Series(list(map(lambda doc: remove.sub(string=doc.lower(), repl=''), documents)))

In [ ]:
a

In [ ]:
clean_train = filter_non_english_chars(train_data.comment_text)
clean_test = filter_non_english_chars(test_data.comment_text)

In [ ]:
def remove_stop_words(texts):
    return list(map(lambda text: ' '.join(map(lambda word: word, text)), 
                map(lambda words: list(filter(lambda word: word not in stop_words, words)), texts.str.split())))

In [ ]:
clean_train = remove_stop_words(clean_train)
clean_test = remove_stop_words(clean_test)

In [ ]:
train_data.comment_text = clean_train
test_data.comment_text = clean_test

In [ ]:
train_data.to_csv('../dataset/preprocessed_train.csv', index=False)
test_data.to_csv('../dataset/preprocessed_test.csv', index=False)

## Length Distribution

In [ ]:
clean_lengths = train_data.comment_text.str.split().str.len()

In [ ]:
clean_lengths.plot()

In [ ]:
clean_lengths.hist()

In [ ]:
clean_lengths.describe()

In [ ]:
clean_lengths_value_counts = clean_lengths.value_counts()

In [ ]:
clean_lengths_value_counts.plot()

In [ ]:
clean_lengths.plot.density()

In [ ]:
clean_lengths.plot.box()

## Word Distributions

In [ ]:
cv = CountVectorizer(stop_words=stop_words)

In [ ]:
words = cv.fit_transform(clean_train)

In [ ]:
names = cv.get_feature_names()

In [ ]:
def get_word_count_dict(term_document_matrix, feature_names):
    """
    """
    counts = np.sum(term_document_matrix, axis=0).A1
    
    return {name: count for name, count in zip(feature_names, counts)}

In [ ]:
def get_sorted_word_counts(count_dict):
    return sorted(count_dict.items(), key=operator.itemgetter(1), reverse=True)


In [ ]:
count_dict = get_word_count_dict(words, names)
sorted_d = get_sorted_word_counts(count_dict)

In [ ]:
%matplotlib inline
counts = pd.DataFrame({'counts': list(map(lambda x: x[1], sorted_d))}, index=list(map(lambda x: x[0], sorted_d)))

In [ ]:
len(counts)

In [ ]:
counts[:20].plot(kind='bar')

In [ ]:
`def clean_output_data(data, name):
    labels = pd.concat([data.toxic, data.severe_toxic, data.obscene, 
           data.threat, data.insult, data.identity_hate], axis=1)
    
    data = train_data[labels.sum(axis=1) > 0].reset_index(drop=True)
    
    
    data.to_csv('data/{}.csv'.format(name), index=False)

In [ ]:
clean_output_data(test_data, 'preprocessed_test')